##### código

In [71]:
from elec_bol20 import *
import elec_bol20.util as ebu

In [72]:

df = ebu.open_combine_2019()
df['D_MAS_CC'] = df['MAS'] - df['CC']
df_full = df[df['VV'] > 0]

In [73]:
path = os.path.join(ebu.DATA_PATH0, '2019', 'percentil_trep.csv')
df_comp = ebu.traductor_df(path, ebu.TRAD_2019_PER_TREP).set_index('ID_MESA')
df_comp = df_comp[~df_comp.index.duplicated()]

In [74]:
df = pd.merge(df_full, df_comp, right_index=True, left_index=True, how='inner')

In [75]:
['ID_RECI', 'HAB', 'CC', 'MAS', '21F', 'PDC', 'VV', 'BL', 'NL', 'X', 'Y',
 'PAIS', 'LAT', 'LON', 'DEN', 'REC', 'MUN', 'BOL', 'P_COMP']
df.columns

Index(['ID_RECI', 'HAB', 'CC', 'MAS', '21F', 'PDC', 'VV', 'BL', 'NL', 'X', 'Y',
       'PAIS', 'LAT', 'LON', 'DEN', 'REC', 'MUN', 'BOL', 'D_MAS_CC', 'P_TREP',
       'PJ_TREP'],
      dtype='object')

In [424]:
R = 80

In [425]:
def single_pred(*, R, df):
    df_12 = df.copy()
    df_1 = df_12[df_12['PJ_TREP'] <= R]
    df_2 = df_12[df_12['PJ_TREP'] > R]

    # %%

    # MAS_p  = ebu.predictor(df_1,df_2,'X','Y','HAB','MAS')
    D_MAS_CC = ebu.predictor(df_1, df_2, 'X', 'Y', 'HAB', 'D_MAS_CC')
    VV_p = ebu.predictor(df_1, df_2, 'X', 'Y', 'HAB', 'VV')
    actual_dmascc = df_12['D_MAS_CC'].sum() / df_12['VV'].sum() * 100

    a = df_1['D_MAS_CC'].sum()
    b = D_MAS_CC.sum()
    c = df_1['VV'].sum()
    d = VV_p.sum()
    calc_dmascc = (a + b) / (c + d) * 100
    return {'pred':calc_dmascc,'actual':actual_dmascc}

def rand_diff(*,df,rr):
    test,train = ebu.partition_df(df,rr)
    MAS_p  = ebu.predictor(train,test,'X','Y','HAB','MAS')
    D_MAS_CC = ebu.predictor(train,test,'X','Y','HAB','D_MAS_CC')
    VV_p = ebu.predictor(train,test,'X','Y','HAB','VV')
    actual_dmascc = df['D_MAS_CC'].sum()/df['VV'].sum()*100

    a = train['D_MAS_CC'].sum()
    b = D_MAS_CC.sum()
    c = train['VV'].sum()
    d = VV_p.sum()
    calc_dmascc = (a + b)/(c + d) * 100
    return actual_dmascc - calc_dmascc


def mc_pred(*,R,r,df,n=2):
    df_R = df[df['PJ_TREP'] <= R].copy()
    
    dif = []
    for i in range(n):
        d = rand_diff(df=df_R,rr=r/100)
    
        dif.append(d)
    return np.std(dif,ddof=1)



In [426]:
_pred = single_pred(R=R,df=df)
calc_dmascc = _pred['pred']
actual_dmascc = _pred['actual']
_st = mc_pred(R=R,r=R,df=df,n=20)

In [427]:
_st

0.04394825100022976

In [428]:
calc_dmascc

10.427188981147662

In [429]:
f = bokeh.plotting.Figure()

In [430]:
df_1 = df_12[df_12['PJ_TREP'] <= R]
dd = df_1[['D_MAS_CC', 'PJ_TREP', 'VV']].set_index('PJ_TREP').sort_index()

In [431]:
ds = dd.cumsum()
ds['d_mas_cc'] = ds['D_MAS_CC'] / ds['VV'] * 100

In [432]:
src = bokeh.models.ColumnDataSource(ds[['d_mas_cc']])

In [433]:

f = bokeh.plotting.Figure(height=400, width=600)
f.line(x='PJ_TREP', y='d_mas_cc', source=src, line_width=3)
f.line(x=[R, 100], y=[ds['d_mas_cc'].iloc[-1], calc_dmascc], color='red', line_width=1)
f.varea(x=[R,100],
        y1=[ds['d_mas_cc'].iloc[-1], calc_dmascc + _st * 3],
        y2=[ds['d_mas_cc'].iloc[-1], calc_dmascc - _st * 3],
        color='red',
        alpha=.5,
#         line_width=1
       )
f.toolbar_location = None
f.y_range.start = -14
f.y_range.end = 14
f.x_range.start = 0
f.x_range.end = 101
bokeh.plotting.output_notebook()
bokeh.plotting.show(f)

Loading BokehJS ...

In [434]:
ds.iloc[-1]

D_MAS_CC    3.661630e+05
VV          4.909866e+06
d_mas_cc    7.457698e+00
Name: 79.99484443923156, dtype: float64